In [1]:
# Results

import time
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imutils
import matplotlib.image as mpimg
from collections import OrderedDict
import pandas as pd

from skimage import io, transform
from math import *
import xml.etree.ElementTree as ET 
import pandas as pd
from skimage.transform import AffineTransform, warp
from skimage.transform import rotate as rotate_transform
from skimage.util import random_noise
from skimage.filters import gaussian
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pickle


class Transforms():
    def __init__(self):
        pass
    
    def rotate(self, image, params):

        angle = params['rotation_range'][0]
        angle = (random.uniform(0,1))*random.choice([-1,1])*angle
        transformation_matrix = torch.tensor([
            [+cos(radians(angle)), -sin(radians(angle))], 
            [+sin(radians(angle)), +cos(radians(angle))]
        ])

        image = rotate_transform(np.array(image), angle = angle, mode = 'edge')

        # PIL expects RGB images to be uint with ranges from 0 to 255 so we have to convert it to a type that PIL can excpect ie a uint from 0 to 255 
        return Image.fromarray((image * 255).astype(np.uint8))

    def translation(self, image,  params):
        image_shape = np.array(image).shape
        ty = random.uniform(params['height_shift_range'][0]*image_shape[0],          
                            params['height_shift_range'][1]*image_shape[0])
        tx = random.uniform(params['width_shift_range'][0]*image_shape[1],
                            params['width_shift_range'][1]*image_shape[1] )

        
        horizontal_shift =  tx*random.choice([-1,1])
        vertical_shift = ty*random.choice([-1,1])
        horizontal_shift_normalised = horizontal_shift/image_shape[1]
        vertical_shift_normalised =  vertical_shift/image_shape[0]

        transform = AffineTransform(translation=(-horizontal_shift,-vertical_shift))

        image = warp(np.array(image),transform,mode='edge')


  
        # PIL expects RGB images to be uint with ranges from 0 to 255 so we have to convert it to a type that PIL can excpect ie a uint from 0 to 255 
        return Image.fromarray((image * 255).astype(np.uint8))
        
    def resize(self, image, img_size):
        image = TF.resize(image, img_size)
        return image

    def zoom(self, image, params):

        img_shape = np.array(image).shape
        zoom = random.uniform(params['zoom_range'][0],params['zoom_range'][1])
        image = TF.resize(image,(int(img_shape[0]*zoom), int(img_shape[1]*zoom)) )
        scale_transform = torch.tensor([[zoom, 0], 
                                        [0, zoom]])

        
        return image

    def color_jitter(self, image):
        color_jitter = transforms.ColorJitter(brightness=0.3, 
                                              contrast=0.3,
                                              saturation=0.3, 
                                              hue=0.1)
        image = color_jitter(image)
        return image


    def __call__(self, image, params, image_size):

        # set checked image and landmark to landmark_ and image_ (this is for making sure we use the last checked tranformed instead of wrongly tranformed to do the following               # tranform)
        
        # -----------------------
        image_ = Image.fromarray(image.copy())

        # -----------------------

        # ZOOM
        image  = self.zoom(image_,  params)
        

        # RESIZE

        image = self.resize(image, (image_size, image_size))

        # ----------------------
        #image_, landmarks_ = self.color_jitter(image_, landmarks_)
        # ----------------------
        
        # ROTATE
        image = self.rotate(image,  params)


        # ----------------------

        image = image
        # ----------------------

        # TRANSLATION
        image= self.translation(image, params)

 
        
        image = TF.to_tensor(image)
        # the following tranform normalises each channel to have a mean at 0.5 and std of 0.5 / NOTE: NOT sure if this is theoreticlly better, should check this
        image = TF.normalize(image, [0.5], [0.5])
        return image

class LandmarksDataset():

    def __init__(self, transform=None,zoom = [1.0 - 0.03258157476873315, 1.0 + 0.03258157476873315], rotation = [22], height_shift= [0,0.03003200603616672], width_shift= [0,0.03003200603616672 ]):

        # targets 0
        filenames1 = os.listdir('D:/Tsetse fly Project/Data/Missing_landmarkwings_L/')
        filenames2 = os.listdir('D:/Tsetse fly Project/Data/Missing_landmarkwings_R/')
        # targets 1
        filenames3 = os.listdir('D:/Tsetse fly Project/Data/tsetsedata_2019_left_commas/images_left/')
    
        self. tranform = transform
        self.zoom = zoom
        self.rotation = rotation
        self.height_shift = height_shift
        self.width_shift = width_shift
        self.image_filenames = []
        self.targets = []
        self.image_size = 244
        self.transform = transform
        self.image_dir = 'D:/Tsetse fly Project/Data/Missing_landmarkwings_L/'
        
        self.image_dir2 = 'D:/Tsetse fly Project/Data/Missing_landmarkwings_R/'
        self.image_dir3 = 'D:/Tsetse fly Project/Data/tsetsedata_2019_left_commas/images_left/'
        self.TransF_ = True

       # ------------------- Append left wings data to dataset class ------------

        for filename in filenames1[:]:
            self.image_filenames.append(os.path.join(self.image_dir, filename))
            self.targets.append(1)

            

        # ------------------ Append flipped right wings data to dataset class-----


        for filename in filenames2[:]:
            self.targets.append(1)
            self.image_filenames.append(os.path.join(self.image_dir2, filename))

        num = len(self.targets.copy())
        for filename in filenames3[:num]:
            self.targets.append(0)
            self.image_filenames.append(os.path.join(self.image_dir3, filename))



        # ----------------------

    def TransF(self):
        self.TransF_ = True
    def NoTransF(self):
        self.TransF_ = False
    def resize(self,size):
        self.image_size = size
    def set_params(self, zoom = [0.95, 0.105], rotation = [10], height_shift= [0,0.05], width_shift= [0,0.05]):
        self.zoom = zoom
        self.rotation = rotation
        self.height_shift = height_shift
        self.width_shift = width_shift
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        params = {'zoom_range': self.zoom, 'rotation_range':self.rotation, 'height_shift_range': self.height_shift, 'width_shift_range': self.width_shift }
        image_ = plt.imread(self.image_filenames[index])
        target = torch.tensor(self.targets[index])

        image = plt.imread(self.image_filenames[index])

        
        if self.transform and self.TransF_:
            
            image = self.transform(image_, params, self.image_size)

        else:
            img_shape = image.copy().shape
            image = Image.fromarray(image)
            image = TF.resize(image, (self.image_size,self.image_size))
       
            image = TF.to_tensor(image)
            # the following tranform normalises each channel to have a mean at 0.5 and std of 0.5 / NOTE: NOT sure if this is theoreticlly better, should check this
            image = TF.normalize(image, [0.5], [0.5])

        return image, target, index#self.image_filenames[index]

DataSet = LandmarksDataset(Transforms())


In [ ]:
im = plt.imread('D:/Tsetse fly Project/Data/tsetsedata_2019_left_commas/images_left/'+ os.listdir(image_dir3)[0])
plt.imshow(im)

In [23]:


class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

'''
You instantiate it with the same arguments used for the normalize. and then use it the same way

unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
unorm(tensor)

'''

'\nYou instantiate it with the same arguments used for the normalize. and then use it the same way\n\nunorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))\nunorm(tensor)\n\n'

In [24]:
#models

class vgg16_bn_(nn.Module):
    def __init__(self,num_classes=1):
        super().__init__()
        self.model_name='vgg16_bn'
        self.model=models.vgg16_bn(pretrained=True)
        #self.model.conv1=nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.classifier=nn.Linear(self.model.classifier[0].in_features, num_classes)
        
    def forward(self, x):
        x = torch.sigmoid(self.model(x))
        return x

class resnet18_(nn.Module):
    def __init__(self,num_classes=1):
        super().__init__()
        self.model_name='resnet18'
        self.model=models.resnet18(pretrained = True)
        self.model.conv1=nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x=torch.sigmoid(self.model(x))
        return x

class inception_(nn.Module):
    def __init__(self,num_classes=1):
        super().__init__()
        self.model_name='inception'
        self.model=models.inception_v3(pretrained=True)
        self.model.Conv2d_1a_3x3.conv=nn.Conv2d(3, 32, kernel_size=3, stride=2,  bias=False)
        self.model.fc=nn.Linear(self.model.fc.in_features, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        if self.model.training:
        
            x = torch.sigmoid(x.logits)
        else:
            
            x = torch.sigmoid(x)
        return x

In [25]:
# split data for image size 244

DataSet.NoTransF()
DataSet.resize(244)
dataset = DataSet
# split the dataset into validation and test sets
len_valid_test_set = int(0.2*len(dataset)) # 60% training, 20% validation, 20% testing

len_train_set = len(dataset) - len_valid_test_set*2

print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_test_set))
print("The length of Valid set is {}".format(len_valid_test_set))

train_dataset , valid_dataset, test_dataset  = torch.utils.data.random_split(dataset , [len_train_set, len_valid_test_set, len_valid_test_set], generator=torch.Generator().manual_seed(42))

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=15, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=15, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)



The length of Train set is 618
The length of Valid set is 205
The length of Valid set is 205


In [28]:
list1 = []
for a,b,i in :
    list1.append(i)
list2=list(set(list1))
if len(list2)==len(list1):
    print('List has unique elements.')
else:
    print('List has duplicate elements.')

List has unique elements.


In [ ]:
#Load vgg16_bn and make prediction on test set

vgg16 = vgg16_bn_()
vgg16.load_state_dict(torch.load('C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/models/model_vgg16_bn_classifer_finetune.pth'))
vgg16.cpu()
vgg16.eval()


start_time = time.time()

y_pred_vgg16 = np.array([])
y_truth_vgg16 = np.array([])
for images,targets in test_loader:
    images = images.reshape((1,3,244,244))
    y = vgg16(images)
    y = y.detach().flatten().numpy()
    y_pred_vgg16 = np.append(y_pred_vgg16, y)
    y_truth_vgg16 = np.append(y_truth_vgg16, targets.detach().flatten().numpy())


endtime = time.time()-start_time
print('time vgg16',endtime)

In [ ]:
#Load resnet18 and make prediction on test set

resnet18 = resnet18_()
resnet18.load_state_dict(torch.load('C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/models/model_resnet18_classifer_finetune2.pth'))
resnet18.cpu()
resnet18.eval()


start_time = time.time()
fp = np.array([])
tp = np.array([])
fn = np.array([])
tn = np.array([])

y_pred_resnet18 = np.array([])
y_truth_resnet18 = np.array([])
for images,targets, imagename in test_loader:
    images = images.reshape((1,3,244,244))
    y = resnet18(images)
    y = y.detach().flatten().numpy()
    y_pred_resnet18 = np.append(y_pred_resnet18, y)
    y_truth_resnet18 = np.append(y_truth_resnet18, targets.detach().flatten().numpy())
    
    if y < 0.197434 :
        y = 0
    else:
        y = 1

    if round(y) == 1 and targets == 0:
        fp = np.append(fp, imagename[-15:])
    elif round(y) == 1 and targets == 1:
        tp = np.append(tp, imagename[-15:])
    elif round(y) == 0 and targets == 1:
        fn = np.append(fn, imagename[-15:])
    elif y == 0 and targets == 0:
        tn = np.append(tn, imagename[-15:])
    

endtime = time.time()-start_time
print('time resnet',endtime)

In [ ]:
h = [fp, tp, fn, tn]

In [ ]:
classifdata = pd.DataFrame([fp, tp, fn, tn])

classifdata = classifdata.T
classifdata.columns = ['fp', 'tp', 'fn', 'tn']
classifdata.to_csv('binaryclassifications.csv')

In [ ]:
for images,targets, imagename in test_loader:
    if imagename[-3:] == 'jpg':
        print(imagename[-26:])
        unorm = UnNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        image = unorm(images)
        image = Image.fromarray(image)
        image = image.permute(1, 2, 0).numpy()
        plt.imshow(image)
        plt.savefig('C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/testset_classif_willieswings/{}'.format(imagename[-26:]))


In [ ]:
fn


In [ ]:
# reload test data with the same seed with image size 299, Load Inception and make prediction on test set

DataSet.NoTransF()
DataSet.resize(299)
dataset = DataSet
# split the dataset into validation and test sets
len_valid_test_set = int(0.2*len(dataset)) # 60% training, 20% validation, 20% testing

len_train_set = len(dataset) - len_valid_test_set*2

print("The length of Train set is {}".format(len_train_set))
print("The length of Valid set is {}".format(len_valid_test_set))
print("The length of Valid set is {}".format(len_valid_test_set))

train_dataset , valid_dataset, test_dataset  = torch.utils.data.random_split(dataset , [len_train_set, len_valid_test_set, len_valid_test_set], generator=torch.Generator().manual_seed(42))

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=15, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=15, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=None, batch_sampler=None,  shuffle=False)

print('evaluating')
inception = inception_()
inception.load_state_dict(torch.load('C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/models/model_inception_classifer_finetune.pth'))
inception.cpu()
inception.eval()

start_time = time.time()
y_pred_inception = []
y_truth_inception = []
for images,targets in test_loader:
    images = images.reshape((1,3,299,299))
    y = inception(images)
    y = y.detach().flatten().numpy()
    y_pred_inception = np.append(y_pred_inception, y)
    y_truth_inception = np.append(y_truth_inception, targets.detach().flatten().numpy())

endtime = time.time()-start_time
print('time inception',endtime)    


In [ ]:
# Classification report for vgg16_bn

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

#
print('vgg16')
vggpred = []
for i in y_pred_vgg16:
    if i<0.5:
        vggpred.append(0)
    else:
        vggpred.append(1)

print(classification_report(y_truth_vgg16, vggpred))
print('AUC ROC:', roc_auc_score(y_truth_vgg16, y_pred_vgg16))


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_truth_vgg16, vggpred)
row_sums = cm.sum(axis=1)
new_matrix = cm / row_sums[:, np.newaxis]
display(new_matrix)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix

print('resnet18')
resnet18pred = []
for i in y_pred_resnet18:
    if i<0.23:
        resnet18pred.append(0)
    else:
        resnet18pred.append(1)

print(classification_report(y_truth_resnet18, resnet18pred))
print('AUC ROC:', roc_auc_score(y_truth_resnet18, y_pred_resnet18))

cm = confusion_matrix(y_truth_resnet18, resnet18pred)
row_sums = cm.sum(axis=1)
new_matrix = cm / row_sums[:, np.newaxis]
display(new_matrix)

In [ ]:
print('inception')
inceptionpred = []
for i in y_pred_inception:
    if i<0.5:
        inceptionpred.append(0)
    else:
        inceptionpred.append(1)

print(classification_report(y_truth_inception, inceptionpred))
print('AUC ROC:', roc_auc_score(y_truth_inception, y_pred_inception))

cm = confusion_matrix( y_truth_inception, inceptionpred )
row_sums = cm.sum(axis=1)
new_matrix = cm / row_sums[:, np.newaxis]
display(new_matrix)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_truth_vgg16, y_pred_vgg16)

import pandas 

vgg16 = pd.DataFrame([fpr, tpr, thresholds])
vgg = vgg16.T.iloc[9:13, :]
vgg.columns = ['fpr', 'tpr', 'thresholds']
vgg

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_truth_inception, y_pred_inception)

import pandas 

vgg16 = pd.DataFrame([fpr, tpr, thresholds])
vgg = vgg16.T.iloc[6:10, :]
vgg.columns = ['fpr', 'tpr', 'thresholds']
vgg

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_truth_resnet18, y_pred_resnet18)

import pandas 

vgg16 = pd.DataFrame([fpr, tpr, thresholds])
vgg = vgg16.T.iloc[6:9, :]
vgg.columns = ['fpr', 'tpr', 'thresholds']
vgg

In [ ]:
print('vgg16')

import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


fpr, tpr, thresholds = metrics.roc_curve(y_truth_vgg16, y_pred_vgg16)

# %%
#plt.figure(figsize=(10,10))
#
plt.subplots(figsize=(10, 6))
plt.plot(fpr, tpr, 'o-', label="ROC curve", linewidth = 1)
#plt.plot(np.linspace(0,1,10), np.linspace(0,1,10), label="diagonal")
for x, y, txt in zip(fpr, tpr, thresholds):
    plt.annotate(np.round(txt,2), (x, y))
#plt.set_fontsize(10)
rnd_idx = 12
plt.annotate('threshold at 0.5', xy = (0.02631579, 0.97368), arrowprops=dict(facecolor='black', lw=2, arrowstyle='->'), xytext = (0.02631579-0.02, 0.9736+0.04))
plt.scatter(0.02631579, 0.9736, color = 'r')
#plt.annotate('this point refers to the tpr and the fpr\n at a probability threshold of {}'.format(np.round(thresholds[rnd_idx], 2)), 
#             xy=(fpr[rnd_idx], tpr[rnd_idx]), xytext=(fpr[rnd_idx]+0.2, tpr[rnd_idx]-0.25),
#             arrowprops=dict(facecolor='black', lw=2, arrowstyle='->'),)
plt.legend(loc="lower right")
plt.title('vgg16_bn ROC curve')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.xlim((0,0.2))
plt.ylim((0.8, 1.1))
plt.savefig('vggroc.png')

In [ ]:
print('inception')

import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


fpr, tpr, thresholds = metrics.roc_curve(y_truth_inception, y_pred_inception)

# %%
plt.subplots(figsize=(10, 6))
plt.plot(fpr, tpr, 'o-', label="ROC curve")
plt.plot(np.linspace(0,1,10), np.linspace(0,1,10), label="diagonal")
for x, y, txt in zip(fpr[::5], tpr[::5], thresholds[::5]):
    plt.annotate(np.round(txt,2), (x, y-0.04))
rnd_idx = 7
plt.annotate('this point refers to the tpr and the fpr\n at a probability threshold of {}'.format(np.round(thresholds[rnd_idx], 2)), 
             xy=(fpr[rnd_idx], tpr[rnd_idx]), xytext=(fpr[rnd_idx]+0.2, tpr[rnd_idx]-0.25),
             arrowprops=dict(facecolor='black', lw=2, arrowstyle='->'),)
plt.legend(loc="lower right")
plt.title('Inception ROC curve')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.savefig('inceproc.png')

In [ ]:
print('inception')

import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


fpr, tpr, thresholds = metrics.roc_curve(y_truth_inception, y_pred_inception)

# %%
plt.subplots(figsize=(10, 6))
plt.plot(fpr, tpr, 'o-', label="ROC curve")
plt.plot(np.linspace(0,1,10), np.linspace(0,1,10), label="diagonal")
for x, y, txt in zip(fpr, tpr, thresholds):
    plt.annotate(np.round(txt,2), (x, y))
#0.97368421, 0.02631579
plt.annotate('threshold at 0.5', xy = (0.02631579,0.974), arrowprops=dict(facecolor='black', lw=2, arrowstyle='->'), xytext = (0.02631579-0.02, 0.97368421+0.04))
plt.scatter(0.02631579,0.979, color = 'r')
plt.legend(loc="lower right")
plt.title('Inception ROC curve')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.xlim((0,0.2))
plt.ylim((0.8, 1.1))
plt.savefig('inceproc.png')

In [ ]:
print('resnet18')

import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


fpr, tpr, thresholds = metrics.roc_curve(y_truth_resnet18, y_pred_resnet18)

# %%
plt.subplots(figsize=(10, 6))
plt.plot(fpr, tpr, 'o-', label="ROC curve")
plt.plot(np.linspace(0,1,10), np.linspace(0,1,10), label="diagonal")
for x, y, txt in zip(fpr[::5], tpr[::5], thresholds[::5]):
    plt.annotate(np.round(txt,2), (x, y-0.04))
rnd_idx = 6
plt.annotate('this point refers to the tpr and the fpr\n at a probability threshold of {}'.format(np.round(thresholds[rnd_idx], 2)), 
             xy=(fpr[rnd_idx], tpr[rnd_idx]), xytext=(fpr[rnd_idx]+0.2, tpr[rnd_idx]-0.25),
             arrowprops=dict(facecolor='black', lw=2, arrowstyle='->'),)
plt.legend(loc="lower right")
plt.title('resnet18 ROC curve')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.savefig('resnetroc.png')

In [ ]:
print('resnet18')

import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt


fpr, tpr, thresholds = metrics.roc_curve(y_truth_resnet18, y_pred_resnet18)

# %%
plt.subplots(figsize=(10, 6))
plt.plot(fpr, tpr, 'o-', label="ROC curve")
plt.plot(np.linspace(0,1,10), np.linspace(0,1,10), label="diagonal")
for x, y, txt in zip(fpr, tpr, thresholds):
    plt.annotate(np.round(txt,2), (x, y))

plt.annotate('threshold at 0.5', xy = ( 0.01754386,0.94174757), arrowprops=dict(facecolor='black', lw=2, arrowstyle='->'), xytext = ( 0.01754386-0.02, 0.94174757+0.04))
plt.scatter( 0.01754386,0.94174757, color = 'r')
plt.legend(loc="lower right")
plt.title('resnet18 ROC curve')
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.xlim((-0.001,0.2))
plt.ylim((0.8, 1.1))
plt.savefig('resnetroc.png')

In [ ]:
print('model training loss')

vgg17 = pickle.load( open( "C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/classifiers/model_vgg16_bn_classifer_finetune_trainingdata.pkl", "rb" ) )
resnet18 = pickle.load(open( "C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/classifiers/model_resnet18_classifer_finetune_trainingdata.pkl", "rb" ))
inception = pickle.load(open( "C:/Users/dylan/Work-Projects/msc_haar/manuscript1_exp/classifiers/model_inception_classifer_finetune_trainingdata.pkl", "rb" ))


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(vgg17['acc_val'], label='vgg16_bn')
plt.plot(resnet18['acc_val'], label='resnet18')
plt.plot(inception['acc_val'], label='inception')
plt.title('Validation accuracies')
plt.xlabel('epoch')
plt.legend()
plt.savefig('valepoch.png')
plt.show()
print(max(vgg17['acc_val']))


In [ ]:
plt.plot(vgg17['loss_val'], label='vgg16_bn')
plt.plot(resnet18['loss_val'], label='resnet18')
plt.plot(inception['loss_val'], label='inception')
plt.title('Validation loss')
plt.legend()
plt.show()

1. create a roc curve for all 3 models

2. set up pipline 